In [1]:
import numpy as np
from keras.models import Sequential
from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import SGD,Adam,RMSprop
from keras.layers.core import Dropout

Using TensorFlow backend.


In [2]:
#input

data = np.zeros((3,32,32))

In [3]:
# model - single output softmax for cropped images
# all activations are currently relu except for output layer which is softmax

model = Sequential()
# C1 layer - convolution size 5 x 5
model.add(Convolution2D(16, 5, 5, input_shape = (32, 32, 3), border_mode = 'valid', activation = 'relu', W_constraint = maxnorm(3)))
# S2 layer - halve dimensions
model.add(MaxPooling2D(pool_size = (2,2)))
# C3 layer - convolution size 5 x 5
model.add(Convolution2D(32, 5, 5, activation = 'relu', border_mode = 'valid', W_constraint = maxnorm(3)))
# S4 layer - halve dimensions
model.add(MaxPooling2D(pool_size = (2,2)))
# C5 layer - convolution size 5 x 5
model.add(Convolution2D(64, 5, 5, activation = 'relu', border_mode = 'valid', W_constraint = maxnorm(3)))
# flatten layer
model.add(Flatten())
# Dense layer
model.add(Dense(16, activation = 'relu', W_constraint = maxnorm(3)))
# Dropout
model.add(Dropout(0.5))
# Output layer
model.add(Dense(10, activation = 'softmax'))

# Compile model
epochs = 25
lrate = 0.01
decay = lrate / epochs
sgd = SGD(lr = lrate,momentum = 0.9,decay = decay,nesterov = False)
model.compile(loss = 'categorical_crossentropy',optimizer = 'sgd',metrics=['accuracy'])
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 28, 28, 16)    1216        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 14, 14, 16)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 10, 10, 32)    12832       maxpooling2d_1[0][0]             
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 5, 5, 32)      0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [8]:
# model - Multiple softmax output

batch_size = 128
num_classes = 10
num_epochs = 25
max_length = 5

model_input = Input(shape=(128,128,3))
c1 = Convolution2D(32, 3, 3, border_mode = 'valid')(model_input)
a1 = Activation('relu')(c1)
s2 = MaxPooling2D(pool_size=(2,2))(a1)
c3 = Convolution2D(32, 3, 3, border_mode = 'valid')(s2)
a3 = Activation('relu')(c3)
s4 = MaxPooling2D(pool_size=(2, 2))(a3)
d = Dropout(0.25)(s4)
conv_out = Flatten()(d)

o0 = Dense(max_length, activation='softmax')(conv_out)
o1 = Dense(num_classes, activation='softmax')(conv_out)
o2 = Dense(num_classes, activation='softmax')(conv_out)
o3 = Dense(num_classes, activation='softmax')(conv_out)
o4 = Dense(num_classes, activation='softmax')(conv_out)
o5 = Dense(num_classes, activation='softmax')(conv_out)

lst = [o0, o1, o2, o3, o4, o5]

modelalt = Model(input=model_input, output=lst)

sgd = SGD(lr = lrate, decay = decay, momentum = 0.9, nesterov = True)
modelalt.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = [custom_metric])
print modelalt.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 128, 128, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_12 (Convolution2D) (None, 126, 126, 32)  896         input_5[0][0]                    
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 126, 126, 32)  0           convolution2d_12[0][0]           
____________________________________________________________________________________________________
maxpooling2d_11 (MaxPooling2D)   (None, 63, 63, 32)    0           activation_9[0][0]               
___________________________________________________________________________________________

In [10]:
# custom metric
import keras.backend as K
from collections import defaultdict

def get_from_one_hot(arr):
    for i in range(10):
        if arr[i] == 1:
            return i

# dimensions of y_true = 6 x num_samples
def custom_metric(y_true, y_pred):
    acc = defaultdict(int)
    for i in range(6):
        for j in range(len(y_true[i])):
            if get_from_one_hot(y_true[i][j]) == get_from_one_hot(y_pred[i][j]):
                acc[i] = acc[i] + 1
        acc[i] = acc[i] / float(len(y_true[i]))
    return {
        'length_accuracy' : acc[0],
        'digit1_accuracy' : acc[1],
        'digit2_accuracy' : acc[2],
        'digit3_accuracy' : acc[3],
        'digit4_accuracy' : acc[4],
        'digit5_accuracy' : acc[5]
    }